# Tape Analysis

In [1]:
import pandas as pd
import datetime


In [3]:
df1 = pd.read_excel('input/cvx.xlsx') #import data from excel file

In [5]:
df1.sort_index(ascending=False, inplace = True) # rotate dataFrame

In [7]:
df1.drop(columns=['Bid','Ask','Sizes'],inplace=True) # delete columns

In [10]:
df1.dropna(how='any',inplace=True) # delete Nan Values

In [33]:
df1.index=df1['Date']

In [34]:
df1

,Date,Time,Price,Size
Date,,,,
2014-03-11,2014-03-11,15:58:55,114.5900,77.0
2014-03-11,2014-03-11,15:58:55,114.5900,23.0
2014-03-11,2014-03-11,15:58:55,114.5900,1.0
2014-03-11,2014-03-11,15:58:55,114.6000,100.0
2014-03-11,2014-03-11,15:58:55,114.6000,100.0
...,...,...,...,...
2014-03-12,2014-03-12,13:17:22,115.5300,100.0
2014-03-12,2014-03-12,13:17:22,115.5300,200.0
2014-03-12,2014-03-12,13:17:22,115.5300,100.0


In [48]:
res_list =[]
pr = 0
dt = 0
tm = 0
sz = 0



In [49]:
df1

,Date,Time,Price,Size
Date,,,,
2014-03-11,2014-03-11,15:58:55,114.5900,77.0
2014-03-11,2014-03-11,15:58:55,114.5900,23.0
2014-03-11,2014-03-11,15:58:55,114.5900,1.0
2014-03-11,2014-03-11,15:58:55,114.6000,100.0
2014-03-11,2014-03-11,15:58:55,114.6000,100.0
...,...,...,...,...
2014-03-12,2014-03-12,13:17:22,115.5300,100.0
2014-03-12,2014-03-12,13:17:22,115.5300,200.0
2014-03-12,2014-03-12,13:17:22,115.5300,100.0


In [65]:
res_list =[]
pr = 0
dt = 0
tm = 0
sz = 0

ls1=[]

for index, row in df1.iterrows():
    if (row['Price']== pr) & (row['Time']==tm) :
        sz += row['Size']
        
    if (row['Price']!= pr):
        #print(index)
        ls1=(dt,tm,pr,sz)
        res_list.append(ls1)
        
        dt = row['Date']
        tm = row['Time']
        pr = row['Price']
        sz = row['Size']


In [64]:
#res_list

In [81]:
df_output = pd.DataFrame(res_list)
df_output.columns= ['Date','Time','Price','Volume']
df_output['Price_1']=0
df_output['Volume_1']=0

In [82]:
pr = 0

for index, row in df_output.iterrows():
    if row['Price']> pr:
        
        df_output.at[index,'Price_1']=row['Price']
        df_output.at[index,'Volume_1']=row['Volume']
        
        
    if row['Price']< pr:
        df_output.at[index,'Price_1']=row['Price']*-1
        df_output.at[index,'Volume_1']=row['Volume']*-1
    pr = row['Price']   
    

In [91]:
df_output.loc['Volume_1'] = df_output.sum(numeric_only=True)

In [92]:
df_output

,Date,Time,Price,Volume,Price_1,Volume_1
0,0,0,0.000000e+00,0.0,0.0000,0.0
1,2014-03-11 00:00:00,15:58:55,1.145900e+02,101.0,114.5900,101.0
2,2014-03-11 00:00:00,15:58:55,1.146000e+02,715.0,114.6000,715.0
3,2014-03-11 00:00:00,15:58:55,1.145900e+02,52.0,-114.5900,-52.0
4,2014-03-11 00:00:00,15:58:55,1.146000e+02,658.0,114.6000,658.0
...,...,...,...,...,...,...
11636,2014-03-12 00:00:00,13:17:15,1.155200e+02,400.0,115.5200,400.0
11637,2014-03-12 00:00:00,13:17:15,1.155150e+02,100.0,-115.5150,-100.0
11638,2014-03-12 00:00:00,13:17:15,1.155200e+02,300.0,115.5200,300.0
11639,2014-03-12 00:00:00,13:17:22,1.155300e+02,416.0,115.5300,416.0


In [84]:
#df['Time']=pd.to_datetime(df["Time"],format='%H:%M:%S')

In [93]:
with pd.ExcelWriter("output/output.xlsx") as writer:
   
    df_output.to_excel(writer, sheet_name="output")
    